# Extended Kalman Filter

Use uncertain information (sensor data) and knowledge of how the system behaves (educated guess) to understand the state of a system which is continuously changing

eg, combine
- an intermittent and imprecise GPS signal
- prediction of what will _probably_ happen when you enter a certain input, such as 'forward'
- noisey but high frequency sensor data from an IMU

To get a very accurate estimate of the current position and velocity of a system

KF assumes that the variables are random and Gaussian distributed with a mean value $\mu$ and variance/uncertainty $\sigma^2$. However, KF relies on the fact that the variables involved are related in some way - eg. position and velocity

Following a guide from https://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/ . Equation numbers correspond with this guide

## The State Estimate

$$ x_k = \begin{bmatrix} \theta \\ \dot{\theta} \end{bmatrix} $$

where $\theta$ is the absolute angle to the object

## The Covariance Matrix

$$ P = \begin{bmatrix} \Sigma_{pp} & \Sigma_{pv} \\ \Sigma_{vp} & \Sigma_{vv} \end{bmatrix}$$

where $ \Sigma_{ij} $ is the degree of variance between the _i'th_ and _j'th_ state variable.

## The Prediction Matrix, Control Matrix and Control Vector

Indicates how we move from one state to the next

$$ p_k = p_{k-1} + \Delta t v_{k-1} + \frac{1}{2} a \Delta t^2 $$
$$ v_k = v_{k-1} + a \Delta t$$

Thus,

$$ \begin{split} \hat{x}_k &= \begin{bmatrix} 1 & \Delta t \\ 0 & 1 \end{bmatrix}\hat{x}_{k-1} + \begin{bmatrix} \frac{\Delta t^2}{2} \\ \Delta t \end{bmatrix} a \\ &= F_k\ \hat{x}_{k-1} + B_k\ u_k \end{split} $$

where $F_k$ is the prediction matrix, $B_k$ is the control matrix, and $u_k$ is the control vector

$B$ and $u$ represented tracked external influences

In other words, the new best estimate is a prediction made from the previous best estimate, plus a correction for known external influences

## Factoring in Uncertainty

Eg. uncertainty from modelling, noise, etc

Update the prediction matrix as follows,

$$ P_k = F_k\ P_{k-1}\ F_k^T + Q_k$$

In other words, the new uncertainty is predicted from the old uncertainty, with some additional uncertainty from the environment

## The Sensor Uncertainty Matrix

$$ \mu_{expected} = H_k\ \hat{x}_k $$
$$ \Sigma_{expected} = H_k\ P_k\ H_k^T $$

where $H_k$ is a matrix which models the sensors

In [ ]:
import numpy as np

In [ ]:
class ExtendedKalmanFilter():
    """
        x       the state estimate, θ and θ_dot
        P       the covariance matrix
        F       the prediction matrix
        Q       the [xxxxx] uncertainty matrix
        R       the [xxxxx] uncertainty matrix
        K       the Kalman gain matrix
    """
    def __init__(self, Ts=0.1, Q_val=10, R_val=1, camera_FOV=62.2):
        self.Ts = Ts
        
        self.x = np.array([0, 0])
        self.P = np.array([[1, 0],
                           [0, 1]])
        self.F = np.array([[1, Ts],
                           [1,  0]])
        self.Q = np.array([[0,     0],
                           [0, Q_val]])
        self.R = np.array([[0,     0],
                           [0, R_val]])
        self.camera_FOV = camera_FOV
        
        
    def predict(self):
        """
            get from x[k-1] to x[k] by using knowledge of how the system behaves 
            x = F*x
            P = F*P*F' + Q
        """
        self.x = self.F @ self.x
        self.P = (self.F @ self.P @ self.F.T) + self.Q
    
    
    def update(self, z):
        """
            get from x[k] to the best possible estimate of the system by using sensor data
            y = z - h(x)
            K = P*H' * inv(H*P*H' + R)
            x = x + K*(z - H*x)
            P = P - K*H*P
            
            where z = the pixel position, scaled from -1 (pixel 0) to 1 (end pixel)
        """
        y = z - self.h(self.x)
        K = (self.P @ self.H.T) @ np.invert(self.H @ self.P @ self.H.T + R)
        
        self.x = self.x  +  K @ y
        self.P = (np.eye(2) - K @ self.H) @ self.P
    
    
    def h(self, x):
        """ convert from angle to pixel
        x is the pixel value [-1, 1]
        cam_angle is the angular size of the camera along that dimension"""
        return np.tan(x[0]) / np.tan(self.camera_FOV/2)
    
    
    def get_predictions(self):
        return self.x

In [ ]:
# EKF = ExtendedKalmanFilter()

In [ ]:
# EKF.predict()
# print('x =', EKF.x)
# print('P =\n', EKF.P)

In [ ]:
# EKF.update(1)